In [638]:
import os
from dotenv import load_dotenv
from numpy import vectorize 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import plotly.graph_objects as go
import string

s=""
sample_files = []
load_dotenv()

Space_Cleared_Exts = os.environ["EXTENSIONS"].replace(" ", "")
# ------------------------------------------------------------------
# Parcourt tout les fichiers finissant avec les extensions dans .env
# ------------------------------------------------------------------
for ext in os.environ["EXTENSIONS"] :
    if(ext==",") :
        ext="["
        for doc in os.listdir("./FilesToTest/") :
            if doc.endswith(str(s)) :
                sample_files.append(''.join(doc))
        s = ""
    if(ext!="[" and ext!="]") :
        s+=ext
for doc in os.listdir("./FilesToTest/") :
    if doc.endswith(str(s.replace(" ",""))) :
        sample_files.append(''.join(doc))
 
sample_contents = [open("./FilesToTest/"+File).read() for File in sample_files]
print(sample_contents)

['var x = math.floor(math.random() * 5);\nb = 2\n\nvar kek = "kek";\n\nif(x==1) {\n    b = 3\n} else {\n    b = 1\n}\n\n\n// DDDDDDDDDDDDdd\n/* HEHEBOY \n\n\n\nSDqsd\n*/', 'Content3\n\n// Normal Commentary', 'yes\n// LE CHOCOLAT\n// SQDJQSDIQSDISDISDISJDISQOQIDQSOIDQSIDJQSIDJSIOJDISQDJIOSQDOIQSDJIQSDOQSJD0IQSDOKSDOIQSJODQSJDOQKDJs', 'HEHE']


In [639]:
Commentary_Content = []
Var_Content = []
Equal_Content = []
Sample_Content_Double = []
Sample_Content_Without_VarComms = []

# ---------------------------------------------------------------------------------
# Parcourt notre texte et enleve les commentaires si le texte vient d'un fichier JS
# ---------------------------------------------------------------------------------

def PurgeJSCommentary(word) :
    Sample_ContentsBis = sample_contents[word].splitlines(True)
    Sample_Content_SubFile = []
    Commentary_SubFile = []
    Long_Comment = False
    for lines in Sample_ContentsBis :
        if(Long_Comment) :
            Commentary_SubFile.append(lines)
        if("//" in lines) :
            Commentary_SubFile.append(lines)
        if("/*" in lines):
            Long_Comment = True
            Commentary_SubFile.append(lines)
        if("*/" in lines):
            Long_Comment = False
            Commentary_SubFile.append(lines)
        else :
            Sample_Content_SubFile.append(lines)
    Sample_Content_Double.append(Sample_Content_SubFile)
    Commentary_Content.append(Commentary_SubFile)

In [640]:
def PurgeJSVariables(index) :
    Sample_Content_SubFile = []
    Var_SubFile = []
    for lines in Sample_Content_Double[index] :
        if("var" in lines) :
            Var_SubFile.append(lines)
        else :
            Sample_Content_SubFile.append(lines)
    Sample_Content_Without_VarComms.append(Sample_Content_SubFile)
    Var_Content.append(Var_SubFile)

In [641]:
def PurgeJSEquals(index) :
    Sample_Content_SubFile =[]
    Equal_SubFile = []
    for lines in Sample_Content_Without_VarComms[index] :
        if("=" in lines) :
            Equal_SubFile.append(lines)
        else :
            Sample_Content_SubFile.append(lines)
    Sample
    Equal_Content.append(Equal_SubFile)

In [642]:
def SwitchArray() :
    sample_contents = Sample_Content_Double
    Sample_Content_Double.clear()

In [643]:
    for i in range(len(sample_files)) :
        if(sample_files[i].endswith(".js")) :
            PurgeJSCommentary(i)
            PurgeJSVariables(i)
            # PurgeJSEquals(i)

In [644]:
print(Commentary_Content)
print(Sample_Content_Without_VarComms)
print(Var_Content)


[['// DDDDDDDDDDDDdd\n', '/* HEHEBOY \n', '\n', '\n', '\n', 'SDqsd\n', '*/', '*/'], ['// Normal Commentary'], ['// LE CHOCOLAT\n', '// SQDJQSDIQSDISDISDISJDISQOQIDQSOIDQSIDJQSIDJSIOJDISQDJIOSQDOIQSDJIQSDOQSJD0IQSDOKSDOIQSJODQSJDOQKDJs']]
[['b = 2\n', '\n', '\n', 'if(x==1) {\n', '    b = 3\n', '} else {\n', '    b = 1\n', '}\n', '\n', '\n', '// DDDDDDDDDDDDdd\n', '/* HEHEBOY \n', '\n', '\n', '\n', 'SDqsd\n'], ['Content3\n', '\n', '// Normal Commentary'], ['yes\n', '// LE CHOCOLAT\n', '// SQDJQSDIQSDISDISDISJDISQOQIDQSOIDQSIDJQSIDJSIOJDISQDJIOSQDOIQSDJIQSDOQSJD0IQSDOKSDOIQSJODQSJDOQKDJs']]
[['var x = math.floor(math.random() * 5);\n', 'var kek = "kek";\n'], [], []]


In [645]:
def tokenize() :
    return ""
tokenize()

''

In [646]:
# vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()

x = TfidfVectorizer()
x.fit_transform(Sample_ContentsBis)
print(x.vocabulary_)
def vectorize(Text) :
    return TfidfVectorizer().fit_transform(Text).toarray()

# similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

def similarity(doc1,doc2) :
    return cosine_similarity([doc1,doc2])

{'var': 12, 'math': 9, 'floor': 4, 'random': 10, 'kek': 7, 'if': 6, 'else': 3, 'dddddddddddddd': 2, 'heheboy': 5, 'sdqsd': 11, 'content3': 1, 'yes': 13, 'le': 8, 'chocolat': 0}


In [647]:

vectors = vectorize(Sample_ContentsBis)
s_vectors = list(zip(sample_files, vectors))

In [648]:

def check_plagiarism():
    results = set()
    global s_vectors
    for sample_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((sample_a, text_vector_a))
        del new_vectors[current_index]
        for sample_b, text_vector_b in new_vectors:
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            sample_pair = sorted((sample_a, sample_b))
            score = sample_pair[0], sample_pair[1], sim_score
            results.add(score)
    return results

In [649]:
for data in check_plagiarism():
    print(data)

('no.js', 'testingfile.js', 0.0)
('carabistouille.c', 'no.js', 0.13675697456974828)
('carabistouille.c', 'yes.js', 0.0)
('testingfile.js', 'yes.js', 0.0)
('no.js', 'yes.js', 0.0)
('carabistouille.c', 'testingfile.js', 0.0)
